<a href="https://colab.research.google.com/github/quetion/time_series_11202/blob/main/TS_Ch4_%E9%A0%90%E6%B8%AC%E8%A1%A8%E7%8F%BE%E4%B9%8B%E8%A9%95%E4%BC%B0_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diebold-Mariano 檢定
- 以比特幣報酬為例
- 比較AR(1)與AR(2)的模型預測能力
- 利用Diebold-Mariano 檢定

In [1]:
# package
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import statsmodels.api as sm
#from statsmodels.regression.rolling import RollingOLS

In [32]:
data = yf.download('BTC-USD',start = '2020-01-01',end='2022-01-01')
data = data[['Adj Close']] # only keep column = ['Adj Close']
data.columns = ['price_btc'] # rename the column
data.index = pd.to_datetime(data.index) # reset the index to be the date format

[*********************100%%**********************]  1 of 1 completed


In [ ]:
data.price_btc.plot()

## 設定自變數
- 報酬率
- 落後一期報酬率
- 落後二期報酬率

- 這裡是建立報酬率
$$
y_t, y_{t-1}
$$
- markdown語法
`the format code`

In [ ]:
# construct the log return (y_t)
data['ret_btc'] = np.log(data.price_btc).diff()
# construct the lagged return (y_{t-1},y_{t-2})
data['ret_btc_1'] = data.ret_btc.shift(1)
data['ret_btc_2'] = data.ret_btc.shift(2)

In [ ]:
# construct AR(1) (in-sample)
y = data.ret_btc
x = sm.add_constant(data.ret_btc_1)
results = sm.OLS(y,x,missing='drop').fit()
print(results.summary())

## 設定樣本範圍
- 資料總數為`1190`
- 樣本內資料設定為`R=200`
- 樣本外資料設定為`P=990`


In [36]:
R = 200 # the number in the sample to construct the model
P = len(data) - R # the number (out-of-sample) to check the performance of the model
print(R,P)


200 531


In [37]:
y = data.ret_btc[1:201]
x = sm.add_constant(data.ret_btc_1[1:201])
result_ar1 = sm.OLS(y,x,missing='drop').fit()

In [48]:
y.head()

Date
2020-01-02   -0.030273
2020-01-03    0.050172
2020-01-04    0.008915
2020-01-05    0.000089
2020-01-06    0.047161
Name: ret_btc, dtype: float64

In [50]:
y[1:5]

Date
2020-01-03    0.050172
2020-01-04    0.008915
2020-01-05    0.000089
2020-01-06    0.047161
Name: ret_btc, dtype: float64

In [18]:
# out-of-sample MSE (mean of squared error)
e1 = np.zeros(P) # AR(1) error
e2 = np.zeros(P) # AR(2) error
for i in range(1,P-1):
    y = data.ret_btc[i:R+i]
    x = sm.add_constant(data.ret_btc_1[i:R+i])
    result_ar1 = sm.OLS(y,x,missing='drop').fit()
    x = sm.add_constant(data[['ret_btc_1','ret_btc_2']][i:R+i])
    result_ar2 = sm.OLS(y,x,missing='drop').fit()
    fitted_ar1 = result_ar1.params[0] + result_ar1.params[1]*data.ret_btc_1[R+i]
    fitted_ar2 = result_ar2.params[0] + result_ar2.params[1]*data.ret_btc_1[R+i] + result_ar2.params[2]*data.ret_btc_2[R+i]
    e1[i] = data.ret_btc[R+i] - fitted_ar1
    e2[i] = data.ret_btc[R+i] - fitted_ar2
mse_ar1_oos = np.mean(e1**2)
mse_ar2_oos = np.mean(e2**2)
print(mse_ar1_oos)
print(mse_ar2_oos)

0.0015026110264478017
0.0015223818612785144


In [19]:
print('AR(1) out-of-sample MSE = {:.4f}'.format(mse_ar1_oos))
print('AR(2) out-of-sample MSE = {:.4f}'.format(mse_ar2_oos))
y = e1**2 - e2**2
x = np.ones(len(y))
# 利用 Newey-West標準誤估計
rr = sm.OLS(y,x,missing='drop').fit(cov_type='HAC',cov_kwds={'maxlags':2})
print(rr.summary())

AR(1) out-of-sample MSE = 0.0015
AR(2) out-of-sample MSE = 0.0015
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 30 Apr 2024   Prob (F-statistic):                nan
Time:                        03:20:47   Log-Likelihood:                 3797.1
No. Observations:                 531   AIC:                            -7592.
Df Residuals:                     530   BIC:                            -7588.
Df Model:                           0                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------

## 判斷模型好壞
- MSE 直接比大小
- 檢定: pvalue

In [20]:
rr.pvalues

array([0.031079])

## 範例練習
- 更換資料頻率成月頻率
  - 根據月頻率資料，建立樣本內AR(1)和AR(2)的模型
  - 設定樣本內資料筆數=30
  - 在AR(1)和AR(2)下，分別建立樣本外的MSE
- 更換標的資產: `S&P500指數`
- 如何匯出下載資料 `.to_csv()`


In [63]:
# 重新獲取資料
data = yf.download('BTC-USD',start = '2018-01-01',end='2024-01-01')
data = data[['Adj Close']]
data.columns = ['price_btc']
data.index = pd.to_datetime(data.index)


[*********************100%%**********************]  1 of 1 completed


In [64]:
# 變更成月頻率
data_m = data.resample('M').last().copy()
# 建立報酬率
data_m['ret_btc'] = np.log(data_m.price_btc).diff()
# 建立落後期
data_m['ret_btc_1'] = data_m.ret_btc.shift()
data_m['ret_btc_2'] = data_m.ret_btc.shift(2)

In [65]:
data_m.head()

,price_btc,ret_btc,ret_btc_1,ret_btc_2
Date,,,,
2018-01-31,10221.099609,NaN,NaN,NaN
2018-02-28,10397.900391,0.017150,NaN,NaN
2018-03-31,6973.529785,-0.399482,0.017150,NaN
2018-04-30,9240.549805,0.281480,-0.399482,0.017150
2018-05-31,7494.169922,-0.209476,0.281480,-0.399482


In [66]:
data_m.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72 entries, 2018-01-31 to 2023-12-31
Freq: M
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   price_btc  72 non-null     float64
 1   ret_btc    71 non-null     float64
 2   ret_btc_1  70 non-null     float64
 3   ret_btc_2  69 non-null     float64
dtypes: float64(4)
memory usage: 2.8 KB


In [67]:
# 樣本內估計
#AR(1)
y = data_m.ret_btc
x = sm.add_constant(data_m.ret_btc_1)
result_ar1 = sm.OLS(y,x,missing='drop').fit()
print(result_ar1.summary())


                            OLS Regression Results                            
Dep. Variable:                ret_btc   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.348
Date:                Tue, 30 Apr 2024   Prob (F-statistic):              0.250
Time:                        03:57:12   Log-Likelihood:                 12.391
No. Observations:                  70   AIC:                            -20.78
Df Residuals:                      68   BIC:                            -16.28
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0174      0.025      0.706      0.4

In [68]:
# 樣本內估計
#AR(2)
y = data_m.ret_btc
x = sm.add_constant(data_m[['ret_btc_1','ret_btc_2']])
result_ar2 = sm.OLS(y,x,missing='drop').fit()
print(result_ar2.summary())

                            OLS Regression Results                            
Dep. Variable:                ret_btc   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                    0.6953
Date:                Tue, 30 Apr 2024   Prob (F-statistic):              0.503
Time:                        03:58:59   Log-Likelihood:                 13.926
No. Observations:                  69   AIC:                            -21.85
Df Residuals:                      66   BIC:                            -15.15
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0235      0.025      0.959      0.3

In [69]:
R = 30 # the number in the sample to construct the model
P = len(data_m) - R # the number (out-of-sample) to check the performance of the model
print(R,P)

30 42


In [70]:
# out-of-sample MSE (mean of squared error)
e1 = np.zeros(P) # AR(1) error
e2 = np.zeros(P) # AR(2) error
for i in range(1,P-1):
    y = data_m.ret_btc[i:R+i]
    x = sm.add_constant(data_m.ret_btc_1[i:R+i])
    result_ar1 = sm.OLS(y,x,missing='drop').fit()
    x = sm.add_constant(data_m[['ret_btc_1','ret_btc_2']][i:R+i])
    result_ar2 = sm.OLS(y,x,missing='drop').fit()
    fitted_ar1 = result_ar1.params[0] + result_ar1.params[1]*data_m.ret_btc_1[R+i]
    fitted_ar2 = result_ar2.params[0] + result_ar2.params[1]*data_m.ret_btc_1[R+i] + result_ar2.params[2]*data_m.ret_btc_2[R+i]
    e1[i] = data_m.ret_btc[R+i] - fitted_ar1
    e2[i] = data_m.ret_btc[R+i] - fitted_ar2
mse_ar1_oos = np.mean(e1**2)
mse_ar2_oos = np.mean(e2**2)
print('AR(1) out-of-sample MSE = {:.4f}'.format(mse_ar1_oos))
print('AR(2) out-of-sample MSE = {:.4f}'.format(mse_ar2_oos))

AR(1) out-of-sample MSE = 0.0393
AR(2) out-of-sample MSE = 0.0419


In [71]:
y = e1**2 - e2**2
x = np.ones(len(y))
# 利用 Newey-West標準誤估計
rr = sm.OLS(y,x,missing='drop').fit(cov_type='HAC',cov_kwds={'maxlags':2})
print(rr.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 30 Apr 2024   Prob (F-statistic):                nan
Time:                        04:14:46   Log-Likelihood:                 119.93
No. Observations:                  42   AIC:                            -237.9
Df Residuals:                      41   BIC:                            -236.1
Df Model:                           0                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0025      0.002     -1.314      0.1